In [ ]:
# Quantum vs Classical SVM on Breast Cancer Dataset
# Author: Z. Staśkowiak
# Description: This notebook compares a Quantum Support Vector Classifier (QSVC)
# using Qiskit's quantum kernel with a classical SVM using scikit-learn.
# ----------------------------------------------------------

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.algorithms import QSVC
import numpy as np
from sklearn.datasets import load_breast_cancer

In [ ]:

# using sklearn's breast cancer dataset
# Load dataset
x, y = load_breast_cancer(return_X_y=True)

# splitting the data into train/test splits

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)


In [ ]:

# normalize
scaler = MaxAbsScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=5)
xs_train = pca.fit_transform(x_train)
xs_test = pca.transform(x_test)

In [12]:
# Adjust the feature map to match the number of features after PCA
features = xs_train.shape[1]

map = ZZFeatureMap(feature_dimension=features, reps=1, entanglement="circular")

sp = Sampler()
fidelity_kind = ComputeUncompute(sampler=sp)


In [13]:
qkernel = FidelityQuantumKernel(fidelity=fidelity_kind, feature_map=map)

# Train QSVM
QSVM = QSVC(quantum_kernel=qkernel)
QSVM.fit(xs_train, y_train)


print("Quantum SVM Accuracy:", accuracy_score(QSVM.predict(xs_test), y_test))

Quantum SVM Accuracy: 0.9298245614035088


In [15]:
# SVM for comparison

SVM = SVC(gamma='scale')
SVM.fit(xs_train, y_train)

print("Classical SVM Accuracy:", accuracy_score(SVM.predict(xs_test), y_test))

Classical SVM Accuracy: 0.9736842105263158
